In [7]:
# -*- coding: utf-8 -*-
"""
골프 스윙 시퀀스 데이터셋 평가 및 예측/정답 CSV 파일 저장
(학습 제외 20% 중 5% 사용)

"""

import os
import json
from PIL import Image
from glob import glob
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from torchvision import transforms
from collections import defaultdict
import random
from sklearn.metrics import accuracy_score
import pandas as pd
import torch.nn as nn

class GolfSwingSequenceDataset(Dataset):
    def __init__(self, root_dir, transform=None, sequence_length=5):
        self.transform = transform
        self.sequence_length = sequence_length
        self.samples = []

        select_task = "test"  # tf는 train, test는 test 데이터셋을 의미
        true_json_dir = os.path.join(root_dir, select_task, "true", "json")
        false_json_dir = os.path.join(root_dir, select_task, "false", "json")
        true_jpg_dir = os.path.join(root_dir, select_task, "true", "jpg")
        false_jpg_dir = os.path.join(root_dir, select_task, "false", "jpg")

        def collect_sequences(json_dir):
            swings = defaultdict(list)
            for fname in os.listdir(json_dir):
                if not fname.endswith(".json"):
                    continue
                swing_id = fname.split("_")[0]  # 예: 001_0001 -> 001
                swings[swing_id].append(fname)
            for key in swings:
                swings[key] = sorted(swings[key])
            return swings

        true_swings = collect_sequences(true_json_dir)
        false_swings = collect_sequences(false_json_dir)

        # ✅ 최대 샘플 수: true/false 중 작은 값 기준
        max_samples = min(len(true_swings), len(false_swings), 20000)

        # ✅ 랜덤하게 swing_id 추출
        true_keys = random.sample(list(true_swings.keys()), max_samples)
        false_keys = random.sample(list(false_swings.keys()), max_samples)

        # ✅ true/false 각각 수집
        balanced_samples = []

        def collect_samples(swings, json_dir, jpg_dir, label):
            sequences = []
            for key in swings:
                file_names = swings[key]
                for i in range(0, len(file_names) - sequence_length + 1):
                    sequence = file_names[i:i + sequence_length]
                    sequences.append({
                        "label": label,
                        "files": [(
                            os.path.join(json_dir, f),
                            os.path.join(jpg_dir, f.replace(".json", ".jpg"))
                        ) for f in sequence]
                    })
            return sequences

        true_samples = collect_samples({k: true_swings[k] for k in true_keys}, true_json_dir, true_jpg_dir, label=1)
        false_samples = collect_samples({k: false_swings[k] for k in false_keys}, false_json_dir, false_jpg_dir, label=0)

        # ✅ 같은 개수로 섞기
        min_len = min(len(true_samples), len(false_samples))
        self.samples = true_samples[:min_len] + false_samples[:min_len]

        # ✅ 무작위로 셔플
        random.shuffle(self.samples)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        images = []
        for json_path, img_path in sample["files"]:
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            images.append(image)
        sequence = torch.stack(images)
        label = torch.tensor(sample["label"], dtype=torch.float32)
        return sequence, label

# ✅ 이미지 전처리 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# ✅ root 디렉토리는 tf보다 상위 디렉토리까지 포함
root_path = "D:/golfDataset/스포츠 사람 동작 영상(골프)/Training/Public/male"

# ✅ 데이터셋 로드
dataset = GolfSwingSequenceDataset(root_dir=root_path, transform=transform, sequence_length=5)

# ✅ 전체 데이터셋을 80% 학습, 20% 평가로 분할
train_size = int(0 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

# ✅ 평가 데이터셋에서 5%만 랜덤 추출
eval_size = int(len(val_dataset))
indices = random.sample(range(len(val_dataset)), eval_size)
eval_dataset = Subset(val_dataset, indices)

# ✅ DataLoader 정의 (평가 데이터셋)
eval_loader = DataLoader(eval_dataset, batch_size=4, shuffle=False)



class CNN_GRU_Classifier(nn.Module):
    def __init__(self, hidden_size=128, num_layers=1):
        super().__init__()
        # CNN encoder
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten()
        )
        self.flattened_size = 32 * 56 * 56  # assuming input is 224x224
        self.gru = nn.GRU(input_size=self.flattened_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):  # x shape: (B, T, C, H, W)
        B, T, C, H, W = x.shape
        x = x.view(B * T, C, H, W)
        x = self.cnn(x)
        x = x.view(B, T, -1)  # reshape for GRU
        out, _ = self.gru(x)
        out = out[:, -1, :]  # 마지막 타임스텝 출력
        out = self.fc(out)
        return out

# ✅ 저장된 모델 로드 (경로는 실제 저장 경로와 일치해야 합니다.)
save_path = r"D:\golfDataset\CNN\cnn_gru_model.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_GRU_Classifier().to(device)
model.load_state_dict(torch.load(save_path))
model.eval()  # 평가 모드로 설정

# ✅ 평가 시작
print("🚀 Evaluation Started")
all_labels = []
all_predictions = []
all_file_paths = []

with torch.no_grad():
    for batch in tqdm(eval_loader, desc="Evaluating"):
        x_batch, y_batch = batch
        x_batch = x_batch.to(device)  # 배치 데이터를 모델과 같은 장치로 이동
        y_batch = y_batch.to(device)
        outputs = model(x_batch)
        probabilities = torch.sigmoid(outputs).cpu().numpy()
        predictions = probabilities.round().astype(int)
        labels = y_batch.cpu().numpy().astype(int)

        all_predictions.extend(predictions)
        all_labels.extend(labels)

        # 파일 경로 수집
        for i_batch in range(x_batch.size(0)): # 각 배치 내의 시퀀스
            file_paths_in_sequence = []
            # 현재 배치 내에서의 인덱스를 사용하여 Subset의 원본 인덱스 접근
            batch_offset = i_batch
            global_index_in_subset = batch_offset
            if global_index_in_subset < len(eval_dataset.indices):
                original_index = eval_dataset.indices[global_index_in_subset]
                sample = dataset.samples[original_index]
                file_paths_in_sequence.append([os.path.basename(f[1]) for f in sample["files"]])
            all_file_paths.append(file_paths_in_sequence)


# ✅ 정확도 계산
accuracy = accuracy_score(all_labels, all_predictions)
print(f"🎯 Evaluation Accuracy: {accuracy:.4f}")
print("✅ Evaluation Finished")

# ✅ 예측과 정답을 DataFrame으로 만들기
eval_df = pd.DataFrame({
    'file_paths': all_file_paths,
    'true_label': all_labels,
    'predicted_label': all_predictions
})

# ✅ CSV 파일로 저장
csv_save_path = "evaluation_results.csv"
eval_df.to_csv(csv_save_path, index=False)
print(f"📝 Evaluation results saved to: {csv_save_path}")

🚀 Evaluation Started


Evaluating: 100%|██████████| 56/56 [00:18<00:00,  2.99it/s]

🎯 Evaluation Accuracy: 1.0000
✅ Evaluation Finished
📝 Evaluation results saved to: evaluation_results.csv
